# Introdução

Uma rede de supermercados (normalmente WallMart) usando mineração de dados descobre que há uma estranha correlacão entre compra de cerveja e compra de fraldas (veja mais sobre isso [aqui](https://www.theregister.com/2006/08/15/beer_diapers/)). Em algumas versões a rede coloca um estande de cerveja ao lado das fraldas.

As técnicas de mineração de regras de associação e de conjunto de intens (itemset) frequentes é que permitem tirar este tipo de conclusão.

Esse notebook surge das minhas anotações pessoais do curso [Pattern Discovery in Data Mining](https://www.coursera.org/learn/data-patterns) junto com materiais complementares (vídeos, artigos, etc) que usei para me ajudar a entender o conteúdo apresentado no curso. Algumas das imagens usadas veem dos slides do curso que mencionei.

# Possibilidades do uso de Regras de Associação

Neste tipo de problema há um conjunto de itens (itens num supermercado) e há transações que contem um subconjunto dos itens (uma compra). Mas podemos abstrair isso e generalizar o uso desse recurso. Seguem algumas possibilidades:

* os itens podem ser paginas num site, a transação as paginas visitadas em diferentes interações com o site.
* o conceito de transação pode não ser localizado no tempo. Pode ser uma pessoa e os itens podem ser aplicativos que essa pessoa instalou no seu celular (não necessariamente ao mesmo tempo).

* pode ser proteinas ativas em diferentes tecidos de diferentes individuos (uma transação é a combinação de tecido e individuo).

# Itemsets frequentes

Itemsets frequentes são conjunto de itens que aparecem juntos em pelo menos *s*% das transações. O número *s*, que precisa ser fornecido para o algoritmo é chamado de suporte.

Vamos assumir as seguintes transações:

* ABC
* AC
* CD
* AB
* BD
* D

Se o suporte foi definido como 1/3, ou seja queremos conjuntos de itens que aparecem em
pelo menos 2 das 6 transações, então AC, AB e D são itemsets frequentes.

Um padrão muito grande de intemset contém uma possibilidade de combinações muito grande. Suponha que tenhamos os seguintes itemsets:

* $T_1= {a_1, ..., a_{50}}$
* $T_2= {a_1, ..., a_{50}, ..., a_{100}}$

As possibilidades de combinações de sub-padrões destes itemsets seriam:

$(\sum_{k=1}^{100}{100\choose k}) - 1= (1 + 1)^{100} -1 = 2^{100} - 1 $

Para encontrar todos os intemsets frequentes, seria necessário computar todas essas possibilidades, o que é impossível de fazer em tempo útil. Portando temos algumas outras definições de itemsets que ajudam a diminuir esse número de possibilidades.

![](imgs/1.png)


Segue as definições de itemsets fechados e maximais:

* um itemset *i* é maximal se ele tem suporte maior que *s* e todos os itemsets que incluem *i* tem suporte menor que s, ou seja, não há super padrão Y frequente em que i é sub-conjunto. No exemplo anterior voltar AB, AC e D são maximais. Essa abordagem perde informações pois sabemos apenas que i é frequente mas não o real suporte do mesmo.


* Um itemset *i* é fechado (closed) se todos os itemsets que o incluem tem suporte menor que *i*.

# A Propriedade de "Fechamento para baixo" dos itemsets frequentes

O Algoritmo Apriori assume que:

"*Se um itemset é frequente, todos seus subitemsets também o serão*"

Isso faz sentido, suponha que o itemset $T_1= {a_1, ..., a_{50}}$ seja frequente, então  ${(a_1), (a_1, a_2), ...}$ também serão frequentes. 

Seguindo esse princípio, deixamos nosso processo de calcular itemsets frequentes mais rápido dado que não precisamos nos preocupar de obter o suporte dos subitemsets de um itemset frequente.

# Apriori: a abordagem Geração de Candidato & Teste

Segue a descrição de cada passo do Algoritmo Apriori:

* inicialmente, varrer todo o Banco de Dados (BD) uma vez em busca de itemsets de tamanho 1 que sejam frequentes
* Repita:
   * Gere itemsets de tamanho (k+1) dos itemsets frequentes de tamanho k
   * Teste os itemsets candidatos no BD para achar itemsets frequentes de tamanho (k+1)
   * faça k := k + 1
* até nenhum conjunto frequente ou candidato puder ser gerado
* retorne todos os itemsets frequentes derivados 

Vamos para uma exemplo concreto.Vamos definir o suporte mínimo como sendo 2 (*minsup = 2*) e que tenhamos um seguinte BD:

![](imgs/2.png)

O primeiro passo seria varrer todo o BD em busca dos itemsets de tamanho 1 que sejam frequentes. Então teríamos:

![](imgs/3.png)

Depois entramos num loop gerando candidatos de tamanho k+1 dos itemsetes frequentes de tamanho k:

![](imgs/4.png)

Fazemos o teste buscando itemsets frequentes aquivalentes aos itemsets candidatos gerados no passo anterior. Os que estão em azul não são frequentes e portanto são eliminados:

![](imgs/5.png)

![](imgs/6.png)

Com esses novos itemsets frequentes de tamanho 2, iremos seguir no loop e vamos gerar itemsets candidatos de tamanho K+1, ou seja, de tamanho 3. Como AB não é frequente, então ABC não será derivado, apenas BCE:

![](imgs/7.png)

Repetimos o processo de varrer o BD buscando itemsets frequentes aquivalentes aos itemsets candidatos gerados no passo anterior e verificamos que o candidato BCE é frequente:

![](imgs/8.png)

Terminamos o processo e então retornamos os itemsets frequentes encontrados.

Um dos passos fudamentais nesse algoritmo é a geração de candidatos. Como gera-los de maneira eficiente? Uma das maneiras seria seguir os passos listados abaixo: 

* fazer o auto-agrupamento (abc + bcd = abcd)
* podagem (quando um subset do auto-agrupamento não existe no BD)

Exemplo:

* $F_3 = {abc, abd, acd, ace, bcd}$
* auto-agrupamento:
   * *abcd* agrupamento que surge das transações *abc* e *abd*
   * *acde* agrupamento que surge das transações *acd* e *ace*
* podagem:
   * *acde* é removido/podado como candidato pois seu subset *ade* não existe em $F_3$


# Apriori: Melhorias e Alternativas

Encontrar o suporte para cada candidato é um processo custoso. Se você tem 10 candidatos, muito provavelmente você terá que varrer todo o BD 10 vezes para encontrar os respectivos suportes.

Seguem algumas abordagem sugeridas para diminuir o custo computacional do Algoritmo Apriori:

* **Reduzir os passos de varredura sobre o BD**:

    * Particionamento (e.g Saravage et al. 1995)
    * Contagem dinâmica de Itemsets (Brin et al. 1997)
    
* **Encolher número de candidatos**:

    * Hashing (e.g, DHP: Park et al, 1995)
    * Podagem por suporte do limear inferior (e.g Bayardo 1998) 
    * Amostragem (e.g Toivonen, 1996)
 
* **Exporar estrutura especiais de dados**:

    * Projeção de Árvore (Aggarwal et al, 2001)
    * Minerador-H (Pei, et al, 2001)
    * Decomposição por Hipercubo (e.g, LCM: Uno, et al, 2004)


### Particionamento: varrer o BD apenas duas vezes

Começamos por um teorema:

"*Qualquer itemset que é potencialmente frequente em um BD de transações (BDT) deve ser frequente em pelo menos uma das partições do  BDT.*"

O método de particionamento consiste em:

* Particionar o BD
* Encontar os itemsets frequentes para cada partição (padrões frequentes locais)
* Consolidar os padrões frequentes globais 

### Hashing Direto e Podagem (DHP)

O DHP (Direct Hashing and Pruning) tem por objetivo reduzir o número de candidatos. Primeiro precisamos entender o que é Hashing Direto e há um vídeo bem curto que explica muito bem a ideia por trás dessa abordagem, assista ele [aqui](https://www.youtube.com/watch?v=SwA_pQH0ihQ).

A ideia é criar uma tabela com buckets contendo itemsets de tamanho k (k-itemsets):

* Cada combinação de k-itemsets candidatos gerado é mapeado para um bucket da Tabela de Hashing Direto e assim a contagem desse bucket é incrementado. 

* Se um bucket, que um k-itemset está associado, não tiver contagem superior a suporte mínimo, o bucket em questão será "podado" e consequentemente os k-itemsets candidatos do bucket em questão não serão considerados frequentes.





### Explorando Dados de Formato Vertical: ECLAT

ECLAT é o acrônimo para "Equivalente Class Transformation" e esse algoritmo tenta explorar as vantagem de dados em formato vertical. 

Um BDT tem o formato horizontal como segue:

![](imgs/10.png)

Mas ele pode ser transformado para um formato vertical como segue:

![](imgs/11.png)

Qual a vantagem disso? Podemos pesquisar hiper-itemsets com base em seus sub-itemsets. Por exemplo:

* $t(e) = {T_{10}, T_{20}, T_{30}}$; 

* $t(a) = {T_{10}, T_{20}}$;

* $t(ae) = t(e) \cap t(a) = {T_{10}, T_{20}}$

Com isso podemos derivar padrões frequentes. Para acelerar ainda mais o processo de minerar padrões frequentes nesse tipo de estrutura de dados podemos monitorar as diferenças ao invés da interseção, pois o resultado da diferença é menor que a interseção de intemsets frequentes, assim salvamos um pouco de memória:

* $ t(ce) = {T_{10}, T_{20}}$

* $ difset(ce) = {T_{20}} $

### FPGrowth: Minerando Padrões Frequentes pelo Crescimento de Padrões

A ideia principal dessa abordagem é que *padrões frequentes crescem*, por isso do nome *Frequent Patterns Growth* (FPGrowth).

Seguem os passos desse algoritmo:

* Encontre itemsets frequentes de tamanho 1 e particione o BD em cada um de tais 1-itemsets frequentes.
* Recursivamente cresça os padrões frequentes aplicando o passo anterior para cada uma dos BDs partiçionados (também conhecidos como BDs particionados)
* Para facilitar a eficiência do processamento, uma estrura de dados eficiente chamada FP-tree pode ser usada (veja mais sobre FP-tree [aqui](https://dzone.com/articles/machinex-understanding-fp-tree-construction#:~:text=To%20put%20it%20simply%2C%20an,items%2C%20their%20paths%20may%20overlap.)).

Usando as FP-tree o processo de mineração de padrões seria o seguinte:

* Recursivamente construa e minere (condicionalmente) as FP-trees.
* Até a FP-tree estar vazia, ou até ela conter um path (paths únicos irão gerar todas suas possíveis combinações de sub-paths, cada um deles sendo padrões frequentes).

As imagens que usarei a seguir para exemplificar a construção de uma FP-trees eu retirei dos slides do curso [Pattern Discovery in Data Mining](https://www.coursera.org/learn/data-patterns).

Vamos supor que temos o seguinte BD de transações:

![](imgs/15.png)

Seguimos os seguintes passos:

* Varrer um BD uma vez para encontrar 1-itemsets frequentes (vamos usar 3 como suporte mínimo). Teremos então: f:4, a:3, c:4, b:3, m:3


Para uma aplicação dessa abordagem usando Python, leia [Understand and Build FP-Growth Algorithm in Python](https://towardsdatascience.com/understand-and-build-fp-growth-algorithm-in-python-d8b989bab342)

